In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.stats import norm, chi2, t, uniform, skewnorm

In [3]:
# define a function to compute the JB statistic, return the statistic and p-value
def JB_statistics(X):
    """
    Input:
        X: A numpy array of shape (N, n) where N is the number of samples 
        and n is sample size
    Output:
        JB_statistic: A numpy array of shape (N,) containing the JB statistic for each sample
        p-value: A numpy array of shape (N,) containing the p-value for each sample
    """
    n = X.shape[1]
    JB_skewness = np.sqrt(n / 6) * skew(X, axis=1, bias=False)
    JB_kurtosis = np.sqrt(n / 24) *kurtosis(X, axis=1, bias=False, \
                                            fisher=True)
    JB_statistic = JB_skewness**2 + JB_kurtosis**2
    p_value = 1 - chi2.cdf(JB_statistic, 2)
    return JB_statistic, p_value

# Define the distributions and their parameters
# create a dictionary as above, but use scipy.stats to generate the random numbers  
distributions = {
    'Normal': lambda size: norm.rvs(size=size),
    'T(3)': lambda size: t.rvs(df=3, size=size),
    'T(10)': lambda size: t.rvs(df=10, size=size),
    'T(30)': lambda size: t.rvs(df=30, size=size),
    'Uniform': lambda size: uniform.rvs(loc=-3, scale=6, size=size),
    'Chi_2(20)': lambda size: chi2.rvs(df=20, size=size),
    'skewed Normal(2)': lambda size: skewnorm.rvs(a=2, loc=0, scale=1, size=size),
    'skewed Normal(-4)': lambda size: skewnorm.rvs(a=-4, loc=0, scale=1, size=size)
}

# Function to generate data from a specified distribution
def generate_data(distribution_name, size):
    if distribution_name in distributions:
        return distributions[distribution_name](size)
    else:
        raise ValueError(f"Distribution '{distribution_name}' is not supported.")



In [4]:
from scipy.stats import norm, jarque_bera, shapiro, normaltest, anderson, kstest
# function to perform normality tests
def perform_normality_tests(data):
    '''
    Input:
        data: A numpy array of shape (N, n) where N is the number of samples 
        and n is sample size
    Output:
        results: A dictionary containing the results of the normality tests
    '''
    results = {}
    # JB statistics
    jb_stat, jb_p = JB_statistics(data)
    results['JB statistics'] = (jb_stat, jb_p)
    # Jarque-Bera test
    jb_stat, jb_p = jarque_bera(data, axis=1)
    results['Jarque-Bera'] = (jb_stat, jb_p)
    # Shapiro-Wilk test
    shapiro_stat, shapiro_p = shapiro(data, axis=1)
    results['Shapiro-Wilk'] = (shapiro_stat, shapiro_p)
    # D'Agostino's K-squared test
    dagostino_stat, dagostino_p = normaltest(data, axis=1)
    results["D'Agostino's K-squared"] = (dagostino_stat, dagostino_p)
    # Anderson-Darling test
    # anderson_result = anderson(data, dist='norm')
    # results['Anderson-Darling'] = anderson_result
    # Kolmogorov-Smirnov test
    ks_stat, ks_p = kstest(data, 'norm', axis=1)
    results['Kolmogorov-Smirnov'] = (ks_stat, ks_p)
    return results

In [13]:
# generate samples from T(3) distribution
np.random.seed(0)
n = 200
N = 10000
X = generate_data('T(3)', (N, n))
JB_stat, JB_p = JB_statistics(X)
JB_power = np.mean(JB_p < 0.05)
print(f"JB Test Power for T(3) distribution: {JB_power}")
jb_stat, jb_p = jarque_bera(X, axis=1)
jb_power = np.mean(jb_p < 0.05)
print(f"Jarque-Bera Test Power for T(3) distribution: {jb_power}")
# results = perform_normality_tests(X)
# print(results)

JB Test Power for T(3) distribution: 0.9921
Jarque-Bera Test Power for T(3) distribution: 0.991
